In [238]:
import numpy as np
import keras.applications
from keras.applications.resnet50 import preprocess_input as p_i, decode_predictions, ResNet50
from keras.preprocessing import image
import h5py
from keras.activations import softmax, relu, sigmoid
#from keras.applications.vgg19 import VGG19
#from keras.applications.vgg19 import preprocess_input
import h5py
from PIL import Image
import gdal
import keras.backend as K
from keras.layers import Dense, Input, Conv2D, Flatten
from keras.layers.core import Dropout, Activation
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.models import Model

In [157]:
K.image_data_format()

'channels_last'

In [189]:
relu

<function keras.activations.relu(x, alpha=0.0, max_value=None, threshold=0.0)>

In [206]:
model = keras.applications.resnet50.ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
#model.layers[-1].activation = soft
model.layers[-1].outbound_nodes = []
model.outputs = [model.layers[-1].output]
output = model.get_layer('fc1000').output
output = Dense(output_dim=1, activation='relu')(output) # your newlayer Dense(...)
model = Model(model.input, output)

/Users/catherinecang/anaconda/envs/gd/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=1)`
  


In [ ]:
def im_to_arr(path):
    ds = gdal.Open(path)
    arr = np.nan_to_num(ds.ReadAsArray())
    return arr

In [123]:
#preprocessing
img_path = '/Users/catherinecang/Downloads/ES705_2005 (5).tif'
im = im_to_arr(img_path).astype(np.uint8)
print(im.shape)
img = Image.fromarray(np.stack(im, axis=2))
img = img.resize((224, 224))
arr = np.array(img)
#img2 = image.array_to_img(im, target_size=(224, 224))
#x = image.img_to_array(img)
#x = preprocess_input(x)


(3, 1655, 1767)


In [212]:
X = np.random.random((100,224,224, 3))#arr.reshape((1, 224, 224, 3))
S = p_i(X)
y = np.ones((100, 1))


In [213]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_17[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [214]:
model.compile(optimizer='rmsprop', loss='mse')

In [230]:
def create_cnn(width, height, depth, filters=(16, 32, 64), regress=True):
	# initialize the input shape and channel dimension, assuming
	# TensorFlow/channels-last ordering
	inputShape = (height, width, depth)
	chanDim = -1

	# define the model input
	inputs = Input(shape=inputShape)

	# loop over the number of filters
	for (i, f) in enumerate(filters):
		# if this is the first CONV layer then set the input
		# appropriately
		if i == 0:
			x = inputs

		# CONV => RELU => BN => POOL
		x = Conv2D(f, (3, 3), padding="same")(x)
		x = Activation("relu")(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = MaxPooling2D(pool_size=(2, 2))(x)

	# flatten the volume, then FC => RELU => BN => DROPOUT
	x = Flatten()(x)
	x = Dense(16)(x)
	x = Activation("relu")(x)
	x = BatchNormalization(axis=chanDim)(x)
	x = Dropout(0.5)(x)

	# apply another FC layer, this one to match the number of nodes
	# coming out of the MLP
	x = Dense(4)(x)
	x = Activation("relu")(x)

	# check to see if the regression node should be added
	if regress:
		x = Dense(1, activation="linear")(x)

	# construct the CNN
	model = Model(inputs, x)

	# return the CNN
	return model

In [233]:
model = create_cnn(224, 224, 3)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [239]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

In [240]:
model.fit(X, y)

Epoch 1/1
100/100 [==============================] - 47s 474ms/step - loss: 94.5022


In [241]:
model.predict(X)

array([[-35.068794],
       [-35.13218 ],
       [-35.09675 ],
       [-34.76563 ],
       [-35.359585],
       [-34.7318  ],
       [-34.517754],
       [-35.51928 ],
       [-35.272945],
       [-35.045403],
       [-35.108925],
       [-34.73377 ],
       [-35.34574 ],
       [-35.032692],
       [-35.297337],
       [-33.579536],
       [-35.4799  ],
       [-35.18426 ],
       [-34.049072],
       [-34.916473],
       [-35.12834 ],
       [-35.28049 ],
       [-35.192184],
       [-35.201305],
       [-34.70326 ],
       [-34.98999 ],
       [-35.03878 ],
       [-35.37736 ],
       [-35.431767],
       [-35.071198],
       [-34.62312 ],
       [-35.425106],
       [-35.227177],
       [-35.208282],
       [-36.014248],
       [-35.477314],
       [-35.191895],
       [-35.118664],
       [-35.36892 ],
       [-35.346916],
       [-34.719448],
       [-35.755302],
       [-34.679173],
       [-34.75607 ],
       [-34.69509 ],
       [-35.596725],
       [-35.194843],
       [-34.8